In [ ]:
#setting os path to import scripts
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# imports
from deep_lens_modeling.network_predictions import generate_sequential_predictions
from scipy.stats import multivariate_normal
from Scripts import network_predictions, fermat_potentials
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines

# lenstronomy stuff
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.Plots import lens_plot

In [ ]:
#autoreload imports
%load_ext autoreload
%autoreload 2

In [ ]:
sample_num= 101
path = module_path+'/'
config_path = 'Configs'
image_path = 'Images_for_Network'
y_test, y_pred_wop, std_pred_wop, prec_pred_wop, y_pred_wp, std_pred_wp, prec_pred_wp, y_pred_wpl, std_pred_wpl, prec_pred_wpl = network_predictions.Predictions(sample_num, path, config_path, image_path)

In [ ]:
# Declare prediction array for all three cases
#pred_arr = [y_pred_wop, y_pred_wp, y_pred_wpl]

In [ ]:
#Finding image positions
x_im_test = []
y_im_test = []

for i in range(sample_num-1):
    x_im_test_val, y_im_test_val = fermat_potentials.image_positions_from_y_pred(y_test[i])
    x_im_test.append(x_im_test_val)
    y_im_test.append(y_im_test_val)

In [ ]:
x_im_pred_wop = []
y_im_pred_wop = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.image_positions_from_y_pred(y_pred_wop[i])
    x_im_pred_wop.append(x_im_pred_val)
    y_im_pred_wop.append(y_im_pred_val)

In [ ]:
x_im_pred_wp = []
y_im_pred_wp = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.image_positions_from_y_pred(y_pred_wp[i])
    x_im_pred_wp.append(x_im_pred_val)
    y_im_pred_wp.append(y_im_pred_val)

In [ ]:
x_im_pred_wpl = []
y_im_pred_wpl = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.image_positions_from_y_pred(y_pred_wpl[i])
    x_im_pred_wpl.append(x_im_pred_val)
    y_im_pred_wpl.append(y_im_pred_val)

In [ ]:
#print(x_im_pred_wp[2])
#print(x_im_test[2])

In [ ]:
#std_pred_wop[0]

In [ ]:
params_dist_wop = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wop[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wop[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wop.append(params_dist_val)

In [ ]:
params_dist_wp = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wp[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wp[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wp.append(params_dist_val)

In [ ]:
params_dist_wpl = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wpl[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wpl[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wpl.append(params_dist_val)

In [ ]:
truth_fermat_potentials = (fermat_potentials.fermat_potential_at_image_positions(y_test[0], x_im_test[0], y_im_test[0]))

In [ ]:
#truth_fermat_arr = []
#for i in range(0,3):
 #   truth_fermat_arr.append(truth_fermat_potentials[0]-truth_fermat_potentials[i+1])
#truth_fermat_arr = np.array(truth_fermat_arr)
arr = np.array([-9, -4, -6])
max_val = max(arr)
min_val = min(arr)
print(arr)
print(max_val)
print(min_val)
abs_max = max(max(arr), min(arr), key=abs)
print(abs_max)
abs_max_loc =  np.where(arr==abs_max)
print(abs_max_loc)

In [ ]:
# Plotting histograms
n = 0
color = ['slategray', 'firebrick', 'goldenrod']
label = ['Without Perturber', 'With Perturber', 'With Perturber Light']
for i in range(sample_num-1):
    sampled_fermat_potentials_wop = []
    sampled_fermat_potentials_wp = []
    sampled_fermat_potentials_wpl = []
    
    for lens_params in params_dist_wop[i]:
        sampled_fermat_potentials_wop.append(fermat_potentials.fermat_potential_at_image_positions(lens_params, x_im_test[i], y_im_test[i]))
    for lens_params in params_dist_wp[i]:
        sampled_fermat_potentials_wp.append(fermat_potentials.fermat_potential_at_image_positions(lens_params, x_im_test[i], y_im_test[i]))
    for lens_params in params_dist_wpl[i]:
        sampled_fermat_potentials_wpl.append(fermat_potentials.fermat_potential_at_image_positions(lens_params, x_im_test[i], y_im_test[i]))

    sampled_fermat_potentials_wop = np.asarray(sampled_fermat_potentials_wop)
    sampled_fermat_potentials_wp = np.asarray(sampled_fermat_potentials_wp)
    sampled_fermat_potentials_wpl = np.asarray(sampled_fermat_potentials_wpl)
    truth_fermat_potentials = (fermat_potentials.fermat_potential_at_image_positions(y_test[i], x_im_test[i], y_im_test[i]))

    #Find location of largest time delay and plot one set of hitograms for each quad
    if sampled_fermat_potentials_wop[i].shape == (4,) and sampled_fermat_potentials_wp[i].shape == (4,) and sampled_fermat_potentials_wpl[i].shape == (4,) and truth_fermat_potentials.shape == (4,):
        
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC','AD']
        truth_arr = []
        sample_wop_arr = []
        sample_wp_arr = []
        sample_wpl_arr = []
        
        for j in range(0,3):
            truth_arr.append(truth_fermat_potentials[0]-truth_fermat_potentials[j+1])
            sample_wop_arr.append(sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,j+1])
            sample_wp_arr.append(sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,j+1])
            sample_wpl_arr.append(sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,j+1])
            
        truth_arr = np.array(truth_arr)
        abs_max = max(max(truth_arr), min(truth_arr), key=abs)
        abs_max_loc =  np.where(truth_arr==abs_max)
        abs_max_loc = int(abs_max_loc[0])
        
        counts,bins,_ = axs.hist(sample_wop_arr[abs_max_loc],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_wp_arr[abs_max_loc],bins=20, histtype=u'step', color='firebrick')
        counts,bins,_ = axs.hist(sample_wpl_arr[abs_max_loc],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr[abs_max_loc],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False, fontsize=5,loc='upper left')
    
    elif sampled_fermat_potentials_wop[i].shape == (3,) and sampled_fermat_potentials_wp[i].shape == (3,) and sampled_fermat_potentials_wpl[i].shape == (3,) and truth_fermat_potentials.shape == (3,):
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC']
        truth_arr = []
        sample_wop_arr = []
        sample_wp_arr = []
        sample_wpl_arr = []

        for j in range(0,2):
            truth_arr.append(truth_fermat_potentials[0]-truth_fermat_potentials[j+1])
            sample_wop_arr.append(sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,j+1])
            sample_wp_arr.append(sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,j+1])
            sample_wpl_arr.append(sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,j+1])
            
        truth_arr = np.array(truth_arr)
        abs_max = max(max(truth_arr), min(truth_arr), key=abs)
        abs_max_loc =  np.where(truth_arr==abs_max)
        abs_max_loc = int(abs_max_loc[0])
        
        counts,bins,_ = axs.hist(sample_wop_arr[abs_max_loc],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_wp_arr[abs_max_loc],bins=20, histtype=u'step', color='firebrick')
        counts,bins,_ = axs.hist(sample_wpl_arr[abs_max_loc],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr[abs_max_loc],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False, fontsize=5,loc='upper left')
    
    elif sampled_fermat_potentials_wop[i].shape == (2,) and sampled_fermat_potentials_wp[i].shape == (2,) and sampled_fermat_potentials_wpl[i].shape == (2,) and truth_fermat_potentials.shape == (2,):
        fig,axs = plt.subplots(1,figsize=(12,4))
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,1],bins=20, histtype=u'step',
                                        color='slategray')
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,1],bins=20, histtype=u'step',
                                        color='firebrick')
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,1],bins=20, histtype=u'step',
                                        color='goldenrod')
        axs.vlines(truth_fermat_potentials[0]-truth_fermat_potentials[1],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{AB}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False,
                fontsize=5,loc='upper left')

    else:
        print('Index:'+str(i))
        print('Sampled fermat potentials WoP size: '+str(sampled_fermat_potentials_wop[i].shape))
        print('Sampled fermat potentials WP size: '+str(sampled_fermat_potentials_wp[i].shape))
        print('Sampled fermat potentials WPL size: '+str(sampled_fermat_potentials_wpl[i].shape))
        print('Truth fermat potentials size: '+str(truth_fermat_potentials.shape))
        n = n+1

print('Missing fermat potential differences: '+str(n))
print(truth_fermat_potentials)